<a href="https://colab.research.google.com/github/toddwalters/pgaiml-python-coding-examples/blob/main/advanced-deep-learning-computer-vision/d7/Perform_Neural_Style_Transfer_Using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perform Neural Style Transfer Using PyTorch

## Step 1: Import the Necessary Libraries
- Install the **torch** package
- Install **torchvision** package which provides utility functions and datasets for working with computer vision tasks in conjunction with PyTorch

**Note:** Install these packages only when using a local machine, not the Simplilearn lab


In [ ]:
#!pip install torch

In [ ]:
#!pip install torchvision

- Import the torch module for working with PyTorch
- Import specific modules from torchvision
- Import the Image module from PIL (Python Imaging Library)
- Import the pyplot module from matplotlib
- Import the numpy module


In [ ]:
import torch
from torchvision import transforms , models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

## Step 2: Check If Cuda Is Available

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

## Step 3: Load a VGG19 Model
- The VGG19 model is loaded with pretrained weights.
- The parameters of the model are set to have **requires_grad** as False, making them non-trainable.
- The model is then moved to the specified device, preparing it for training or inference on that device.

In [ ]:
model = models.vgg19(pretrained=True).features
for p in model.parameters():
    p.requires_grad = False
model.to(device)

## Step 4: Choose Layers for Style and Content Loss
- Add a batch dimension to the input tensor
- Pass the input through the layer
- Store the output of specific layers in the features dictionary



In [ ]:
def model_activations(input,model):
    layers = {
    '0' : 'conv1_1',
    '5' : 'conv2_1',
    '10': 'conv3_1',
    '19': 'conv4_1',
    '21': 'conv4_2',
    '28': 'conv5_1'
    }
    features = {}
    x = input
    x = x.unsqueeze(0)
    for name,layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features

## Step 5: Transform the Images
- Use (0.5, 0.5, 0.5) for both mean and sd

In [ ]:
transform = transforms.Compose([transforms.Resize(300),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

## Step 6: Open the Content Image and Convert It to RGB Format
- Apply transformations to the content image and move it to the specified device
- Print the shape of the content image tensor
- Open the style image and convert it to RGB format
- Apply transformations to the style image and move it to the specified device


In [ ]:
content = Image.open("content.jpg").convert("RGB")
content = transform(content).to(device)
print("Content shape => ", content.shape)
style = Image.open("style.jpg").convert("RGB")
style = transform(style).to(device)

## Step 7: Function to Convert the Image
- Convert the image tensor to a numpy array on the CPU, and remove singleton dimensions
- Transpose the dimensions to bring the channel dimension to the last axis
- Scale the values and shift them to the range [0, 1]
- Clip the values to the range [0, 1]


In [ ]:
def imcnvt(image):
    x = image.to("cpu").clone().detach().numpy().squeeze()
    x = x.transpose(1,2,0)
    x = x*np.array((0.5,0.5,0.5)) + np.array((0.5,0.5,0.5))
    x = np.clip(x, 0.0, 1.0)

    return x

## Step 8: Print the Image
- Create a figure and two subplots
- Display the content image on the first subplot using imshow
- Display the style image on the second subplot using imshow
- Show the figure with the subplots



In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2)

ax1.imshow(imcnvt(content),label = "Content")
ax2.imshow(imcnvt(style),label = "Style")
plt.show()

## Step 9: Get the Dimensions of the Image Feature Tensor
- Reshape the tensor to have dimensions (d, h * w)
- Compute the Gram matrix by matrix multiplication




In [ ]:
def gram_matrix(imgfeature):
    _,d,h,w = imgfeature.size()
    imgfeature = imgfeature.view(d,h*w)
    gram_mat = torch.mm(imgfeature,imgfeature.t())

    return gram_mat

## Step 10: Set Device to Cuda If Available

In [ ]:
target = content.clone().requires_grad_(True).to(device)

print("device = ",device)

## Step 11: Extract the Style and Content Features Using the Model Activations
- The style features are extracted using the **model_activations** function on the style image.
- The content features are extracted using the **model_activations** function on the content image.



In [ ]:
style_features = model_activations(style,model)
content_features = model_activations(content,model)

style_wt_meas = {"conv1_1" : 1.0,
                 "conv2_1" : 0.8,
                 "conv3_1" : 0.4,
                 "conv4_1" : 0.2,
                 "conv5_1" : 0.1}

style_grams = {layer:gram_matrix(style_features[layer]) for layer in style_features}

## Step 12: Extract the Features of the Target Image Using the Model Activations
- Get the style Gram matrix for the layer
- Get the target Gram matrix for the layer
- Compute the Gram matrix for the target
- Compute the style loss for the layer
- Compute the total loss and print the total loss for every tenth epoch
- Clear the gradients
- Backpropagate the total loss
- Update the target image using the optimizer
- Display the target image after a certain number of epochs
- Save the target image as a PNG file



In [ ]:
content_wt = 100
style_wt = 1e8

print_after = 500
epochs = 2000
optimizer = torch.optim.Adam([target],lr=0.007)

for i in range(1,epochs+1):
    target_features = model_activations(target,model)
    content_loss = torch.mean((content_features['conv4_2']-target_features['conv4_2'])**2)

    style_loss = 0
    for layer in style_wt_meas:
        style_gram = style_grams[layer]
        target_gram = target_features[layer]
        _,d,w,h = target_gram.shape
        target_gram = gram_matrix(target_gram)

        style_loss += (style_wt_meas[layer]*torch.mean((target_gram-style_gram)**2))/d*w*h

    total_loss = content_wt*content_loss + style_wt*style_loss

    if i%10==0:
        print("epoch ",i," ", total_loss)

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if i%print_after == 0:
        plt.imshow(imcnvt(target),label="Epoch "+str(i))
        plt.show()
        plt.imsave(str(i)+'.png',imcnvt(target),format='png')

**Observation:**
- The code performs neural style transfer using a VGG19 model, displaying the stylized image every 500 epochs and saving it with the respective epoch number as the filename.